In [47]:
%matplotlib qt5
from process_files import *
from func_science import *

In [53]:
#darks data
iline = 0
line = 'Fe_I_6173'
# calib data
config = configobj.ConfigObj('config.ini')
dkdir = config['darks']['directory']
scdir = config['science']['directory']
bbi_scale = config['targetplate']['bbi_scale']
settings = [f for f in os.listdir(scdir) if 'settings' in f]
settings = scdir + os.sep + settings[0]
settings = configobj.ConfigObj(settings)
# dark frames
dk0 = data_cube(dkdir, line, 0, 0 )
dk0m = process_dark_cube(dk0.data)
dk1 = data_cube(dkdir, line, 1, 0)
dk1m = process_dark_cube(dk1.data)
dk2 = data_cube(dkdir, line, 2, 0)
dk2m = process_dark_cube(dk2.data)

Settings loaded from  D:\Darks\20220302\1239\HELLRIDE_20220339_123950561_settings.ini
Data loading from  D:\Darks\20220302\1239\Fe_I_6173\HELLRIDE_bbi_20220302_123927508_dk.DAT
Settings loaded from  D:\Darks\20220302\1239\HELLRIDE_20220339_123950561_settings.ini
Data loading from  D:\Darks\20220302\1239\Fe_I_6173\HELLRIDE_pol1_20220302_123927683_dk.DAT
Settings loaded from  D:\Darks\20220302\1239\HELLRIDE_20220339_123950561_settings.ini
Data loading from  D:\Darks\20220302\1239\Fe_I_6173\HELLRIDE_pol2_20220302_123927860_dk.DAT


In [54]:
# Other numbers
linestr = 'Line_' + str(get_line_num(settings, line, iline))
nacc = int(settings[linestr]['Polarimeter\\NAccumulations'])
nwav = int(settings[linestr]['NWavePoints'])
filtstr = settings[linestr]['Filter']
modstr = settings[linestr]['Polarimeter\\Modulation']
nmod = int(settings[filtstr]['Polarimeter\\'+modstr+'\\NModulations'])
nfpc = nmod*nacc*nwav
nfpw = nmod*nacc
ncyc = len(os.listdir(scdir+os.sep+line))//3
bbi_scale = np.float64(config['targetplate']['bbi_scale'])
# Time stamps
tsfile = [f for f in os.listdir(scdir) if 'timestamp' in f]
tsfile = scdir + os.sep + tsfile[0]
ts = np.loadtxt(tsfile, delimiter=',')
im0ind, im1ind, im2ind = ts[:,3], ts[:,6], ts[:,9]
im0ind = im0ind[0:nfpc*ncyc].reshape([ncyc,nfpc])
im1ind = im1ind[0:nfpc*ncyc].reshape([ncyc,nfpc])
im2ind = im2ind[0:nfpc*ncyc].reshape([ncyc,nfpc])
#
im0ind = im0ind - im0ind[:,0:1]
im1ind = im1ind - im1ind[:,0:1]
im2ind = im2ind - im2ind[:,0:1]
#
im0ind = im0ind.reshape([ncyc, nwav, nacc, nmod])%nmod
im1ind = im1ind.reshape([ncyc, nwav, nacc, nmod])%nmod
im2ind = im2ind.reshape([ncyc, nwav, nacc, nmod])%nmod

In [55]:
# Get flats
ff0 = pf.open(config['flats'][line+'/bbi'])
ff0m = ff0[0].data[:,:,np.newaxis]
ff0m /= ff0m.mean()
ff0.close()
#
ff1 = pf.open(config['flats'][line+'/pol1'])
continuum1 = ff1[0].data
line_shifts1 = ff1[1].data
ff1m = ff1[2].data
ff1.close()
#
ff2 = pf.open(config['flats'][line+'/pol2'])
continuum2 = ff2[0].data
line_shifts2 = ff2[1].data
ff2m = ff2[2].data
ff2.close()

In [56]:
# Get science data
for i in tqdm.tqdm(range(1)):
    # bbi cam
    dc0 = data_cube(scdir, line, 0, i)
    Y = X = dc0.roi
    sc0 = dc0.data - dk0m
    sc0m = np.sum(sc0.reshape([Y,X,nacc*nmod,nwav], order='F'), axis=2)
    sc0corr = sc0m/ff0m
    # pol1 cam
    dc1 = data_cube(scdir, line, 1, i)
    sc1 = np.mean(dc1.data.reshape([Y,X,nacc*nmod,nwav], order='F'), axis=2) - dk1m
    sc1sh = shift_3darray(sc1, line_shifts1)
    sc1corr = np.fliplr(sc1sh)
    # sc1corr = sc1sh/continuum1
    # pol2 cam
    dc2 = data_cube(scdir, line, 2, i)
    sc2 = np.mean(dc2.data.reshape([Y,X,nacc*nmod,nwav], order='F'), axis=2) - dk2m
    sc2sh = shift_3darray(sc2, line_shifts2)
    sc2corr = np.flipud(np.fliplr(sc2sh))
    

  0%|          | 0/1 [00:00<?, ?it/s]

Settings loaded from  D:\Science\20220302\1317\HELLRIDE_20220317_131730726_settings.ini
Data loading from  D:\Science\20220302\1317\Fe_I_6173\HELLRIDE_bbi_20220302_131732198_sc.DAT
Settings loaded from  D:\Science\20220302\1317\HELLRIDE_20220317_131730726_settings.ini
Data loading from  D:\Science\20220302\1317\Fe_I_6173\HELLRIDE_pol1_20220302_131732544_sc.DAT


100%|██████████| 1280/1280 [01:56<00:00, 10.96it/s]


Settings loaded from  D:\Science\20220302\1317\HELLRIDE_20220317_131730726_settings.ini
Data loading from  D:\Science\20220302\1317\Fe_I_6173\HELLRIDE_pol2_20220302_131732905_sc.DAT


100%|██████████| 1/1 [04:24<00:00, 264.67s/it]


In [59]:
show_img_series(sc1corr, fps=1)

In [45]:
# crop
bbi_crop = 30
bbi_extent = (dc0.roi-2*bbi_crop)*bbi_scale
print('BBI RoI: ', bbi_extent)
bbi_extent = [0,bbi_extent,0,bbi_extent]
fig, ax = plt.subplots(1,1, figsize=(5,5))
ax.imshow(sc0corr[bbi_crop:-bbi_crop,bbi_crop:-bbi_crop,-1], extent=bbi_extent,cmap='gray')
ax.set_xlabel('arcsec')
ax.set_ylabel('arcsec')
fig.tight_layout()
plt.savefig('broadband.png', dpi=1200)

BBI RoI:  90.28


In [46]:
pol2_crop = 70
pol2_extent = (dc2.roi-2*pol2_crop)*bbi_scale*np.float64(config['targetplate']['mag'])
print('Pol2 RoI: ', pol2_extent)
pol2_extent = [0,pol2_extent,0,pol2_extent]
fig, ax = plt.subplots(1,1, figsize=(5,5))
ax.imshow(sc2corr[pol2_crop:-pol2_crop,pol2_crop:-pol2_crop,10], extent=pol2_extent, cmap='gray')
ax.set_xlabel('arcsec')
ax.set_ylabel('arcsec')
fig.tight_layout()
plt.savefig('halpha_core.png', dpi=1200)

Pol2 RoI:  90.70738988381406
